<a href="https://colab.research.google.com/github/Luis-Yovera/Deteccion_infraccion_vehicular_de_estacionamiento_en_cruce_peatonal_durante_semaforo_en_rojo/blob/TAREAS_U1_PDSeI/Copia_de_Pr%C3%A1ctica_PDSeI_01_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Trabajo 01 PDSeI - Galarreta, Recuenco, Yovera - Mecatrónica VIII UNT

## Instalaciones

In [1]:
!pip install torch
!pip install torchvision #Conjuntos de datasets predefinidos y populares como CIFAR10, ImageNet, y COCO. facilitan el procesamiento y la manipulación de imágenes antes de entrenar modelos.

## Importar dependencias

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

## Definir Hiperparámetros

In [3]:
input_size = 784
hidden_size = 256  # Aumentado de 128 a 256
num_classes = 10
num_epochs = 20  # Aumentado de 5 a 20
batch_size = 64  # Ajustado de 100 a 64
lr = 5e-3  # Aumentado de 1e-3 a 5e-3

## Descargando la base de datos mnist

In [4]:
train_data = dsets.FashionMNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)  #Contiene el conjunto de imágenes y etiquetas del conjunto de entrenamiento
                                                                              #  Contiene imágenes de prendas de ropa  //   28x28 píxeles
test_data = dsets.FashionMNIST(root = './data', train = False,
                       transform = transforms.ToTensor()) #Contiene el conjunto de imágenes y etiquetas del conjunto de prueba

100%|██████████| 26421880/26421880 [00:02<00:00, 11542677.52it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 210234.82it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3876706.33it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5790366.58it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



## Leyendo la data

In [5]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)            #Es un DataLoader que gestiona los datos de entrenamiento, dividiéndolos en lotes de tamaño batch_size y barajando las muestras aleatoriamente para cada época.

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)                  # Es Es un DataLoader que gestiona los datos de prueba, también dividiéndolos en lotes de tamaño batch_size, pero sin barajar los datos.

## Definir modelo

In [6]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.1) # Mientras más bajo sea, el modelo podrá retener ayor información
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.drop(out)
    out = self.fc2(out)
    return out

## Instancia del modelo

In [7]:
net = Net(input_size, hidden_size, num_classes)

if torch.cuda.is_available():
  net.cuda()

## Compilación

In [8]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

## Entrenamiento

In [9]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
        print('Epoca [%d/%d], Step [%d/%d], Loss: %.4f'
              % (epoch + 1, num_epochs, i + 1, len(train_data) // batch_size, loss.item()))


Epoca [1/20], Step [100/937], Loss: 0.6391
Epoca [1/20], Step [200/937], Loss: 0.4794
Epoca [1/20], Step [300/937], Loss: 0.4310
Epoca [1/20], Step [400/937], Loss: 0.4955
Epoca [1/20], Step [500/937], Loss: 0.3971
Epoca [1/20], Step [600/937], Loss: 0.4756
Epoca [1/20], Step [700/937], Loss: 0.4153
Epoca [1/20], Step [800/937], Loss: 0.7596
Epoca [1/20], Step [900/937], Loss: 0.5926
Epoca [2/20], Step [100/937], Loss: 0.3392
Epoca [2/20], Step [200/937], Loss: 0.3953
Epoca [2/20], Step [300/937], Loss: 0.2983
Epoca [2/20], Step [400/937], Loss: 0.3400
Epoca [2/20], Step [500/937], Loss: 0.4548
Epoca [2/20], Step [600/937], Loss: 0.6880
Epoca [2/20], Step [700/937], Loss: 0.4314
Epoca [2/20], Step [800/937], Loss: 0.3329
Epoca [2/20], Step [900/937], Loss: 0.3869
Epoca [3/20], Step [100/937], Loss: 0.3459
Epoca [3/20], Step [200/937], Loss: 0.5057
Epoca [3/20], Step [300/937], Loss: 0.4291
Epoca [3/20], Step [400/937], Loss: 0.4500
Epoca [3/20], Step [500/937], Loss: 0.2913
Epoca [3/20

In [10]:
correct = 0
total = 0
for images, labels in test_gen:
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    output = net(images)
    _, predicted = torch.max(output, 1)
    correct += (predicted == labels).sum()
    total += labels.size(0)

print('Accuracy: %.3f %%' % (100 * correct / (total + 1)))

Accuracy: 86.861 %
